# 1) Setup

In [71]:
#import libraries
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm

import numpy as np
import math
import scipy as sp
from scipy import stats
from copy import copy
import time
import yaml
import networkx as nx

import pymongo
from pymongo import MongoClient
from datetime import datetime
from datetime import timedelta
print("done")

done


In [72]:
#import functions from python files

#1) data import & filtering
from get_data import *
from filter_data1 import *
from filter_data2 import *
from filter_data3 import *
from select_data import *
from select_data_edge import *

#2) Bayesian: lognormal
from Gamma import *
from Lognormal import *

from Gaussian import *
from Gaussian_broadcast import *

from Normal_Gamma import *
from Normal_Gamma_bayes import *
from Normal_Gamma_bayes_update import *

#for comparison
from Gaussian_ml import *

from error_mae import *
from error_mse import *

from integrate_pdf import *
from error_ks import *                 #Kolmogorov-Smirnov
from error_ks_2samples import *
from error_cvm import *                #Cramer Von-Mises

#3) Bayesian: Inverse Gamma - method 1
from Inverse_Gamma import *
from Inverse_Gamma_bayes import *
from Inverse_Gamma_bayes2 import *

from Inverse_Digamma_bounds import *
from Inverse_Digamma_newton import *

#4) Bayesian: Inverse Gamma - method 2
from Inverse_Gamma_broadcast import *
from Inverse_Gamma_prior import *
from Inverse_Gamma_bayes3 import *

#5) Bayesian: Inverse Gaussian
from Inverse_Gaussian import *
from Inverse_Gaussian_broadcast import *
from Inverse_Gaussian_prior import * 
from Inverse_Gaussian_bayes import * 

#6) Comparing edges
from underscore_prefix import *
from underscore_suffix import *
from ks_between_edges import *
from get_context import *
from augment_with_context import *

#7) Network visualisation
from draw_graph import *


print("done")

done


In [73]:
# load from excel (see FILTER.ipynb)
df = pd.read_excel("filtered_data_random.xlsx")
df = df.loc[:,["origin", "target", "edge_id", "operation_time"]]
    
print("done")

done


In [74]:
names = pd.read_csv("waypoint_pairs.csv")
names.columns = ["edge_id","count"]
wp_list = []
for i in range(10):
    wp1 = underscore_prefix(names["edge_id"][i])
    wp2 = underscore_suffix(names["edge_id"][i])
    wp_list.append([wp1,wp2])
print(wp_list)

[['WayPoint70', 'WayPoint71'], ['WayPoint71', 'WayPoint70'], ['WayPoint68', 'WayPoint69'], ['WayPoint69', 'WayPoint70'], ['WayPoint71', 'WayPoint72'], ['WayPoint72', 'WayPoint71'], ['WayPoint75', 'WayPoint68'], ['WayPoint69', 'WayPoint68'], ['WayPoint70', 'WayPoint69'], ['WayPoint53', 'WayPoint46']]


In [75]:
#initialise empty dataframe
pd.options.mode.chained_assignment = None  # default='warn'
gof = pd.DataFrame(index = np.arange(10), columns = ["model", "KS", "CVM", "time (s)"] )
print("done")

done


# 2) Lognormal

## 2a) Lognorm - no offset

In [76]:
lognorm_ks = []
lognorm_cvm = []

start_time = time.time()
for i in range(len(wp_list)):
    # Select data of interest
    wp1 = wp_list[i][0]
    wp2 = wp_list[i][1]
    edge = wp1 + "_" + wp2
    subset = select_data_edge(df, edge)
    #independent variable to plot over
    t_op = subset["operation_time"]
    
    #use offset and take log of data
    #offset = np.min(t_op) - 0.001
    offset = 0
    t_log = np.log(t_op - offset)

    #n_terms = 1 if you want just the initial estimates
    n_terms = len(t_log)    
    t_obs = t_log[0:n_terms]


    #set parameters
    mu_0 = 1
    beta = 0.1
    a = 1
    b = 1

    #Bayesian MAP estimate of mean & variance of Gaussian distribution
    tic = time.time()
    mean_test, var_test, posterior, mean_map, var_map = Normal_Gamma_bayes(t_obs, mu_0, beta, a, b, plot_graph = False)
    toc = time.time()
    
    #Duration distribution using MAP parameters from Bayesian method
    t_start= 0.01
    t_stop = (np.max(t_op)//5)*5 + 10    #round up to nearest 5 secs
    t_step = 0.01
    t_test = np.arange(t_start,t_stop,t_step)
    p_bayes = Lognormal(t_test,mean_map,var_map)
    
    #ks test comparison
    ks_bayes = error_ks(t_test+offset,p_bayes, t_op, plot_graph = False)
    lognorm_ks.append(ks_bayes)
    
    #cvm test comparison
    cvm_bayes = error_cvm(t_test+offset,p_bayes, t_op, plot_graph = False)
    lognorm_cvm.append(cvm_bayes)
    
    print(i, ks_bayes, cvm_bayes, toc-tic, "secs")

stop_time = time.time()

print("KS:", np.mean(lognorm_ks),"CVM:", np.mean(lognorm_cvm))
print("Total time:", stop_time - start_time)
    

MAP probability above threshold
0 0.2823025552198209 0.1313599324888886 1.3101258277893066 secs
1 0.344197404600194 0.1378648980409555 0.9809815883636475 secs
2 0.30988652188288396 0.278131831606512 7.080411434173584 secs
MAP probability above threshold
3 0.33467709663169243 0.12504375408365054 1.4177956581115723 secs
MAP probability above threshold
4 0.3393086789783673 0.12578262704692383 1.2879395484924316 secs
5 0.3968987376460351 0.18613038810919255 0.8289308547973633 secs
MAP probability above threshold
6 0.22521763171043635 0.11339520211111047 3.1958212852478027 secs
7 0.2807693612045997 0.07112249388236566 1.8730576038360596 secs
8 0.41033965494946 0.1395013026705073 1.200467586517334 secs
MAP probability above threshold
9 0.316667113615927 0.08085726565858542 1.614471435546875 secs
KS: 0.3240264756439417 CVM: 0.1389189695698692
Total time: 20.850754261016846


In [77]:
gof["model"][0] = "lognorm"
gof["KS"][0] = np.mean(lognorm_ks)
gof["CVM"][0] = np.mean(lognorm_cvm)
gof["time (s)"][0] = stop_time - start_time
gof.head()

,model,KS,CVM,time (s)
0,lognorm,0.324026,0.138919,20.8508
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN


## 2b) Lognorm - offset

In [78]:
lognorm_offset_ks = []
lognorm_offset_cvm = []

start_time = time.time()
context = get_context("walmart_map.yaml")
for i in range(len(wp_list)):
    # Select data of interest
    wp1 = wp_list[i][0]
    wp2 = wp_list[i][1]
    edge = wp1 + "_" + wp2
    subset = select_data_edge(df, edge)
    #independent variable to plot over
    t_op = subset["operation_time"]
    
    #use offset and take log of data
    length = context.loc[context["edge_id"] == edge]["edge_length"]
    offset= np.min([float(length / (0.55 * 1.1) ), np.min(t_op) - 0.01])
    t_log = np.log(t_op - offset)

    #n_terms = 1 if you want just the initial estimates
    n_terms = len(t_log)    
    t_obs = t_log[0:n_terms]


    #set parameters
    mu_0 = 1
    beta = 0.1
    a = 1
    b = 1

    #Bayesian MAP estimate of mean & variance of Gaussian distribution
    tic = time.time()
    mean_test, var_test, posterior, mean_map, var_map = Normal_Gamma_bayes(t_obs, mu_0, beta, a, b, plot_graph = False)
    toc = time.time()
    
    #Duration distribution using MAP parameters from Bayesian method
    t_start= 0.01
    t_stop = (np.max(t_op)//5)*5 + 10    #round up to nearest 5 secs
    t_step = 0.01
    t_test = np.arange(t_start,t_stop,t_step)
    p_bayes = Lognormal(t_test,mean_map,var_map)
    
    #ks test comparison
    ks_bayes = error_ks(t_test+offset,p_bayes, t_op, plot_graph = False)
    lognorm_offset_ks.append(ks_bayes)
    
    #cvm test comparison
    cvm_bayes = error_cvm(t_test+offset,p_bayes, t_op, plot_graph = False)
    lognorm_offset_cvm.append(cvm_bayes)
    
    print(i, ks_bayes, cvm_bayes, toc-tic, "secs")

stop_time = time.time()

print("KS:", np.mean(lognorm_offset_ks),"CVM:", np.mean(lognorm_offset_cvm))
print("Total time:", stop_time - start_time)
    

0 0.041042181752258766 0.016157871634490756 0.1197199821472168 secs
1 0.006820774110695511 0.09562097371374417 0.045908451080322266 secs
2 0.08560676847502219 0.07276121564951393 5.662340402603149 secs
3 0.005818607076202784 0.011262247125110347 0.19647884368896484 secs
4 0.011574093207326308 0.025469265830084263 0.06283092498779297 secs
5 0.012319454599014155 0.1638842253742557 0.009971857070922852 secs
6 0.0921394719928253 0.027601297082756805 36.63324236869812 secs
7 0.10013469696108945 0.013385148067201947 0.6681828498840332 secs
8 0.03992734224982545 0.18178692657756246 0.01997852325439453 secs
9 0.03125705418106872 0.012776078717485862 8.3900728225708 secs
KS: 0.04266404446053286 CVM: 0.06207052497722062
Total time: 52.554640769958496


In [79]:
ind = 1
gof["model"][ind] = "lognorm offset"
gof["KS"][ind] = np.mean(lognorm_offset_ks)
gof["CVM"][ind] = np.mean(lognorm_offset_cvm)
gof["time (s)"][ind] = stop_time - start_time
gof.head()

,model,KS,CVM,time (s)
0,lognorm,0.324026,0.138919,20.8508
1,lognorm offset,0.042664,0.0620705,52.5546
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN


# 3) Inverse Gaussian

## 3a) Invgauss

In [80]:
invgauss_ks = []
invgauss_cvm = []

start_time = time.time()
for i in range(len(wp_list)):
    # Select data of interest
    wp1 = wp_list[i][0]
    wp2 = wp_list[i][1]
    edge = wp1 + "_" + wp2
    subset = select_data_edge(df, edge)
    #independent variable to plot over
    t_op = subset["operation_time"]

    #Plot against actual data
    #offset = np.min(t_op) - 0.001
    offset = 0
    t_obs = t_op - offset

    #define hyperparameters
    a = 1
    b = 1
    c = 1
    d = 1

    #Bayesian MAP estimate
    tic = time.time()
    mu_map,L_map = Inverse_Gaussian_bayes(t_obs, a, b, c, d, threshold = 0.9)
    toc = time.time()
    
    #Duration distribution using MAP parameters from Bayesian method
    t_start= 0.01
    t_stop = (np.max(t_op)//5)*5 + 10    #round up to nearest 5 secs
    t_step = 0.01
    t_test = np.arange(t_start,t_stop,t_step)
    p_bayes = Inverse_Gaussian(t_test, mu = mu_map, L = L_map)
    
    #ks test comparison
    ks_bayes = error_ks(t_test+offset,p_bayes, t_op, plot_graph = False)
    invgauss_ks.append(ks_bayes)
    
    #cvm test comparison
    cvm_bayes = error_cvm(t_test+offset,p_bayes, t_op, plot_graph = False)
    invgauss_cvm.append(cvm_bayes)
    
    print(i, ks_bayes, cvm_bayes, toc-tic, "secs")

stop_time = time.time()

print("KS:", np.mean(invgauss_ks),"CVM:", np.mean(invgauss_cvm))
print("Total time:", stop_time - start_time)
    

0 0.7688970473099079 2.3576543981322975 0.5010082721710205 secs
1 0.8772125820822233 4.108471595359839 0.18948841094970703 secs
2 0.6427416506638043 1.5252036302690448 0.7613377571105957 secs
3 0.7332127812834963 1.940853544657686 0.5824720859527588 secs
4 0.7875862450844255 2.5023258910543014 0.42220330238342285 secs
5 0.8904168456256539 4.356955954792762 0.17353558540344238 secs
6 0.39646604454544937 0.3429465763636809 6.109595060348511 secs
7 0.8236095245292923 3.2314359511021045 0.33062028884887695 secs
8 0.8537678637804775 3.4500964158535172 0.24634099006652832 secs
9 0.462452715846053 0.47536841876329244 3.2415566444396973 secs
KS: 0.7236363300750784 CVM: 2.4291312376348526
Total time: 12.6110258102417


In [81]:
ind = 2
gof["model"][ind] = "invgauss"
gof["KS"][ind] = np.mean(invgauss_ks)
gof["CVM"][ind] = np.mean(invgauss_cvm)
gof["time (s)"][ind] = stop_time - start_time
gof.head()

,model,KS,CVM,time (s)
0,lognorm,0.324026,0.138919,20.8508
1,lognorm offset,0.042664,0.0620705,52.5546
2,invgauss,0.723636,2.42913,12.611
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN


## 3b) Invgauss - offset

In [82]:
invgauss_offset_ks = []
invgauss_offset_cvm = []

start_time = time.time()
context = get_context("walmart_map.yaml")

for i in range(len(wp_list)):
    # Select data of interest
    wp1 = wp_list[i][0]
    wp2 = wp_list[i][1]
    edge = wp1 + "_" + wp2
    subset = select_data_edge(df, edge)
    #independent variable to plot over
    t_op = subset["operation_time"]

    #Plot against actual data
    length = context.loc[context["edge_id"] == edge]["edge_length"]
    offset= np.min([float(length / (0.55 * 1.1) ), np.min(t_op) - 0.01])
    #offset = 0
    t_obs = t_op - offset

    #define hyperparameters
    a = 1
    b = 1
    c = 1
    d = 1

    #Bayesian MAP estimate
    tic = time.time()
    mu_map,L_map = Inverse_Gaussian_bayes(t_obs, a, b, c, d, threshold = 0.9)
    toc = time.time()
    
    #Duration distribution using MAP parameters from Bayesian method
    t_start= 0.01
    t_stop = (np.max(t_op)//5)*5 + 10    #round up to nearest 5 secs
    t_step = 0.01
    t_test = np.arange(t_start,t_stop,t_step)
    p_bayes = Inverse_Gaussian(t_test, mu = mu_map, L = 1/L_map)
    
    #ks test comparison
    ks_bayes = error_ks(t_test+offset,p_bayes, t_op, plot_graph = False)
    invgauss_offset_ks.append(ks_bayes)
    
    #cvm test comparison
    cvm_bayes = error_cvm(t_test+offset,p_bayes, t_op, plot_graph = False)
    invgauss_offset_cvm.append(cvm_bayes)
    
    print(i, ks_bayes, cvm_bayes, toc-tic, "secs")

stop_time = time.time()

print("KS:", np.mean(invgauss_offset_ks),"CVM:", np.mean(invgauss_offset_cvm))
print("Total time:", stop_time - start_time)
    

0 0.6504913235225523 0.33944810872927056 0.4215843677520752 secs
1 0.4980676148856855 0.14882870208432972 0.186492919921875 secs
2 0.5331504949681937 0.2209186421006142 0.7528057098388672 secs
3 0.7114230418661088 0.371297746945732 0.583946943283081 secs
4 0.6897711292985074 0.3119211901560048 0.4138357639312744 secs
5 0.536640576881217 0.1626712386964329 0.1735379695892334 secs
6 0.5869289056694095 0.25827337434953324 5.717601537704468 secs
7 0.698607998068342 0.4839012399189185 0.32515859603881836 secs
8 0.5975023007716553 0.14038096900589153 0.24930167198181152 secs
9 0.6443540619929535 0.29857272007473257 3.0519986152648926 secs
KS: 0.6146937447924625 CVM: 0.27362139320614604
Total time: 12.626843929290771


In [83]:
ind = 3
gof["model"][ind] = "invgauss offset"
gof["KS"][ind] = np.mean(invgauss_offset_ks)
gof["CVM"][ind] = np.mean(invgauss_offset_cvm)
gof["time (s)"][ind] = stop_time - start_time
gof.head()

,model,KS,CVM,time (s)
0,lognorm,0.324026,0.138919,20.8508
1,lognorm offset,0.042664,0.0620705,52.5546
2,invgauss,0.723636,2.42913,12.611
3,invgauss offset,0.614694,0.273621,12.6268
4,NaN,NaN,NaN,NaN


# 4) Invgamma Approximate

## 4a) Invgamma Approximate

In [84]:
invgamma_ks = []
invgamma_cvm = []

start_time = time.time()
for i in range(len(wp_list)):
    # Select data of interest
    wp1 = wp_list[i][0]
    wp2 = wp_list[i][1]
    edge = wp1 + "_" + wp2
    subset = select_data_edge(df, edge)
    #independent variable to plot over
    t_op = subset["operation_time"]

    #Plot against actual data
    #offset = np.min(t_op) - 0.001
    offset = 0
    t_obs = t_op - offset

    #prior hyperparameters
    a = 0.1
    b = 1
    c = 2
    d = 0.3
    e = 1

    #Bayesian MAP estimate
    tic = time.time()
    alpha, beta = Inverse_Gamma_bayes2(t_obs, w1 = 1, w2 = 1)
    toc = time.time()
    
    #Duration distribution using MAP parameters from Bayesian method
    t_start= 0.01
    t_stop = (np.max(t_op)//5)*5 + 10    #round up to nearest 5 secs
    t_step = 0.01
    t_test = np.arange(t_start,t_stop,t_step)
    p_bayes = Inverse_Gamma(t_test, alpha, beta)
    
    #ks test comparison
    ks_bayes = error_ks(t_test+offset,p_bayes, t_op, plot_graph = False)
    if np.isnan(ks_bayes) == False:
        invgamma_ks.append(ks_bayes)
    
    #cvm test comparison
    cvm_bayes = error_cvm(t_test+offset,p_bayes, t_op, plot_graph = False)
    if np.isnan(ks_bayes) == False:
        invgamma_cvm.append(cvm_bayes)
    
    print(i, ks_bayes, cvm_bayes, toc-tic, "secs")

stop_time = time.time()

print("KS:", np.mean(invgamma_ks),"CVM:", np.mean(invgamma_cvm))
print("Total time:", stop_time - start_time)
    

count: 2
0 nan nan 0.0019943714141845703 secs
count: 2
1 nan nan 0.0009877681732177734 secs
count: 2
2 nan nan 0.0009970664978027344 secs
count: 2
3 nan nan 0.0019974708557128906 secs
count: 2
4 nan nan 0.0020189285278320312 secs
count: 2
5 nan nan 0.0020112991333007812 secs
count: 205
6 0.19748366637299847 0.08986297052179579 0.09175539016723633 secs
count: 2
7 nan nan 0.001994609832763672 secs
count: 2
8 nan nan 0.001994609832763672 secs
count: 2
9 nan nan 0.0009970664978027344 secs
KS: 0.19748366637299847 CVM: 0.08986297052179579
Total time: 0.1715407371520996


C:\Users\pyliu\Documents\GOALS\Code\Wk4\Inverse_Gamma_bayes2.py:53: RuntimeWarning: invalid value encountered in log
  k1 = n*( np.mean( -np.log(x) ) - sp.special.digamma(alpha) + np.log(n*alpha) - np.log(np.sum(1/x)) - alpha * sp.special.polygamma(1,alpha) + 1)


In [85]:
ind = 4
gof["model"][ind] = "approx invgamma"
gof["KS"][ind] = np.mean(invgamma_ks)
gof["CVM"][ind] = np.mean(invgamma_cvm)
gof["time (s)"][ind] = stop_time - start_time
gof.head()

,model,KS,CVM,time (s)
0,lognorm,0.324026,0.138919,20.8508
1,lognorm offset,0.042664,0.0620705,52.5546
2,invgauss,0.723636,2.42913,12.611
3,invgauss offset,0.614694,0.273621,12.6268
4,approx invgamma,0.197484,0.089863,0.171541


## 4b) Invgamma Approximate - offset

In [86]:
invgamma_ks = []
invgamma_cvm = []

start_time = time.time()
context = get_context("walmart_map.yaml")
for i in range(len(wp_list)):
    # Select data of interest
    wp1 = wp_list[i][0]
    wp2 = wp_list[i][1]
    edge = wp1 + "_" + wp2
    subset = select_data_edge(df, edge)
    #independent variable to plot over
    t_op = subset["operation_time"]

    #Plot against actual data
    length = context.loc[context["edge_id"] == edge]["edge_length"]
    offset= np.min([float(length / (0.55 * 1.1) ), np.min(t_op) - 0.01])
    #offset = 0
    t_obs = t_op - offset

    #prior hyperparameters
    a = 0.1
    b = 1
    c = 2
    d = 0.3
    e = 1

    #Bayesian MAP estimate
    tic = time.time()
    alpha, beta = Inverse_Gamma_bayes2(t_obs, w1 = 1, w2 = 1)
    toc = time.time()
    
    #Duration distribution using MAP parameters from Bayesian method
    t_start= 0.01
    t_stop = (np.max(t_op)//5)*5 + 10    #round up to nearest 5 secs
    t_step = 0.01
    t_test = np.arange(t_start,t_stop,t_step)
    p_bayes = Inverse_Gamma(t_test, alpha, beta)
    
    #ks test comparison
    ks_bayes = error_ks(t_test+offset,p_bayes, t_op, plot_graph = False)
    if np.isnan(ks_bayes) == False:
        invgamma_ks.append(ks_bayes)
    
    #cvm test comparison
    cvm_bayes = error_cvm(t_test+offset,p_bayes, t_op, plot_graph = False)
    if np.isnan(ks_bayes) == False:
        invgamma_cvm.append(cvm_bayes)
    
    print(i, ks_bayes, cvm_bayes, toc-tic, "secs")

stop_time = time.time()

print("KS:", np.mean(invgamma_ks),"CVM:", np.mean(invgamma_cvm))
print("Total time:", stop_time - start_time)
    

count: 97
0 0.18686140568351178 0.008964381414753024 0.03690147399902344 secs
count: 2
1 nan nan 0.0019943714141845703 secs
count: 40
2 0.11901437762060527 0.02750048698008925 0.01802968978881836 secs
count: 117
3 0.12196898758728025 0.00140006388755392 0.04388260841369629 secs
count: 2
4 nan nan 0.001995086669921875 secs
count: 2
5 nan nan 0.001994609832763672 secs
count: 10
6 0.10515127753156978 0.01623822222403526 0.004986763000488281 secs
count: 78
7 0.12096437780148941 0.007949346776831945 0.02991962432861328 secs
count: 2
8 nan nan 0.000997304916381836 secs
count: 25
9 0.08250346433204336 0.0062617132836160945 0.010970592498779297 secs
KS: 0.12274398175941663 CVM: 0.011385702427813248
Total time: 0.9050083160400391


C:\Users\pyliu\Documents\GOALS\Code\Wk4\Inverse_Gamma_bayes2.py:53: RuntimeWarning: invalid value encountered in log
  k1 = n*( np.mean( -np.log(x) ) - sp.special.digamma(alpha) + np.log(n*alpha) - np.log(np.sum(1/x)) - alpha * sp.special.polygamma(1,alpha) + 1)


In [87]:
ind = 5
gof["model"][ind] = "approx invgamma offset"
gof["KS"][ind] = np.mean(invgamma_ks)
gof["CVM"][ind] = np.mean(invgamma_cvm)
gof["time (s)"][ind] = stop_time - start_time
gof.head(10)

,model,KS,CVM,time (s)
0,lognorm,0.324026,0.138919,20.8508
1,lognorm offset,0.042664,0.0620705,52.5546
2,invgauss,0.723636,2.42913,12.611
3,invgauss offset,0.614694,0.273621,12.6268
4,approx invgamma,0.197484,0.089863,0.171541
5,approx invgamma offset,0.122744,0.0113857,0.905008
6,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN


## 4c) Approx Invgamma - cutoff

In [88]:
invgamma_ks = []
invgamma_cvm = []

start_time = time.time()
for i in range(len(wp_list)):
    # Select data of interest
    wp1 = wp_list[i][0]
    wp2 = wp_list[i][1]
    edge = wp1 + "_" + wp2
    subset = select_data_edge(df, edge)
    #independent variable to plot over
    t_op = subset["operation_time"]

    #Plot against actual data
    #offset = np.min(t_op) - 0.001
    offset = 0
    t_obs = t_op - offset

    #prior hyperparameters
    a = 0.1
    b = 1
    c = 2
    d = 0.3
    e = 1

    #Bayesian MAP estimate
    tic = time.time()
    alpha, beta = Inverse_Gamma_bayes2(t_obs, w1 = 1, w2 = 1)
    toc = time.time()
    
    #Duration distribution using MAP parameters from Bayesian method
    t_start= 0.01
    t_stop = (np.max(t_op)//5)*5 + 10    #round up to nearest 5 secs
    t_step = 0.01
    t_test = np.arange(t_start,t_stop,t_step)
    p_bayes = Inverse_Gamma(t_test, alpha, beta)
    
    p_bayes[t_test<np.min(t_op)] = 0
    #normalise
    spacing = t_test[1]-t_test[0]
    norm_const = spacing * np.sum(p_bayes)
    p_bayes /= norm_const
    
    #ks test comparison
    ks_bayes = error_ks(t_test+offset,p_bayes, t_op, plot_graph = False)
    if np.isnan(ks_bayes) == False:
        invgamma_ks.append(ks_bayes)
    
    #cvm test comparison
    cvm_bayes = error_cvm(t_test+offset,p_bayes, t_op, plot_graph = False)
    if np.isnan(ks_bayes) == False:
        invgamma_cvm.append(cvm_bayes)
    
    print(i, ks_bayes, cvm_bayes, toc-tic, "secs")

stop_time = time.time()

print("KS:", np.mean(invgamma_ks),"CVM:", np.mean(invgamma_cvm))
print("Total time:", stop_time - start_time)
    

count: 2
0 nan nan 0.0009970664978027344 secs
count: 2
1 nan nan 0.000997304916381836 secs
count: 2
2 nan nan 0.0009970664978027344 secs
count: 2
3 nan nan 0.000997304916381836 secs
count: 2
4 nan nan 0.0009970664978027344 secs
count: 2
5 nan nan 0.0019953250885009766 secs
count: 205
6 0.057637325965899475 0.11327068688920743 0.08580279350280762 secs
count: 2
7 nan nan 0.001994609832763672 secs
count: 2
8 nan nan 0.001994609832763672 secs
count: 2
9 nan nan 0.001994609832763672 secs
KS: 0.057637325965899475 CVM: 0.11327068688920743
Total time: 0.16358375549316406


C:\Users\pyliu\Documents\GOALS\Code\Wk4\Inverse_Gamma_bayes2.py:53: RuntimeWarning: invalid value encountered in log
  k1 = n*( np.mean( -np.log(x) ) - sp.special.digamma(alpha) + np.log(n*alpha) - np.log(np.sum(1/x)) - alpha * sp.special.polygamma(1,alpha) + 1)


In [89]:
ind = 6
gof["model"][ind] = "approx invgamma cutoff"
gof["KS"][ind] = np.mean(invgamma_ks)
gof["CVM"][ind] = np.mean(invgamma_cvm)
gof["time (s)"][ind] = stop_time - start_time
gof.head(10)

,model,KS,CVM,time (s)
0,lognorm,0.324026,0.138919,20.8508
1,lognorm offset,0.042664,0.0620705,52.5546
2,invgauss,0.723636,2.42913,12.611
3,invgauss offset,0.614694,0.273621,12.6268
4,approx invgamma,0.197484,0.089863,0.171541
5,approx invgamma offset,0.122744,0.0113857,0.905008
6,approx invgamma cutoff,0.0576373,0.113271,0.163584
7,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN


# 5) Exact Invgamma
- Product of conjugate priors

## 5a) Exact invgamma

In [90]:
invgamma_ks = []
invgamma_cvm = []

start_time = time.time()
for i in range(len(wp_list)):
    # Select data of interest
    wp1 = wp_list[i][0]
    wp2 = wp_list[i][1]
    edge = wp1 + "_" + wp2
    subset = select_data_edge(df, edge)
    #independent variable to plot over
    t_op = subset["operation_time"]

    #Plot against actual data
    #offset = np.min(t_op) - 0.001
    offset = 0
    t_obs = t_op - offset

    #prior parameters
    a = 1
    b = 1
    c = 1
    d = 1
    e = 1

    #Bayesian MAP estimate
    tic = time.time()
    alpha_map, beta_map = Inverse_Gamma_bayes3(t_obs, a, b, c, d, e)
    toc = time.time()
    
    #Duration distribution using MAP parameters from Bayesian method
    t_start= 0.01
    t_stop = (np.max(t_op)//5)*5 + 10    #round up to nearest 5 secs
    t_step = 0.01
    t_test = np.arange(t_start,t_stop,t_step)
    p_bayes = Inverse_Gamma(t_test, alpha= alpha_map, beta = beta_map)
    
    #ks test comparison
    ks_bayes = error_ks(t_test+offset,p_bayes, t_op, plot_graph = False)
    if np.isnan(ks_bayes) == False:
        invgamma_ks.append(ks_bayes)
    
    #cvm test comparison
    cvm_bayes = error_cvm(t_test+offset,p_bayes, t_op, plot_graph = False)
    if np.isnan(ks_bayes) == False:
        invgamma_cvm.append(cvm_bayes)
    
    print(i, ks_bayes, cvm_bayes, toc-tic, "secs")

stop_time = time.time()

print("KS:", np.mean(invgamma_ks),"CVM:", np.mean(invgamma_cvm))
print("Total time:", stop_time - start_time)
    

0 0.6532961823967934 1.3951445500229338 0.9969019889831543 secs
1 0.7427216255765942 2.1447187997498407 0.42890381813049316 secs
2 0.5310252434506534 1.0893137596080016 1.5799248218536377 secs
3 0.6285334709461985 1.2126789202381814 1.2368426322937012 secs
4 0.6699895593030153 1.4478954364733005 0.9016556739807129 secs
5 0.7551112127011952 2.2423128403163535 0.3989264965057373 secs
6 0.3567545846314362 0.26436694701334024 12.579211473464966 secs
7 0.6928616927394149 1.6944517374223702 0.6951727867126465 secs
8 0.7240598028743281 1.8328054510219258 0.544532299041748 secs
9 0.40885793465078646 0.3549485106572868 6.846492052078247 secs
KS: 0.6163211309270415 CVM: 1.3678636952523537
Total time: 26.26335906982422


In [91]:
ind = 7
gof["model"][ind] = "exact invgamma"
gof["KS"][ind] = np.mean(invgamma_ks)
gof["CVM"][ind] = np.mean(invgamma_cvm)
gof["time (s)"][ind] = stop_time - start_time
gof.head(10)

,model,KS,CVM,time (s)
0,lognorm,0.324026,0.138919,20.8508
1,lognorm offset,0.042664,0.0620705,52.5546
2,invgauss,0.723636,2.42913,12.611
3,invgauss offset,0.614694,0.273621,12.6268
4,approx invgamma,0.197484,0.089863,0.171541
5,approx invgamma offset,0.122744,0.0113857,0.905008
6,approx invgamma cutoff,0.0576373,0.113271,0.163584
7,exact invgamma,0.616321,1.36786,26.2634
8,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN


## 5b) Exact Invgamma - offset

In [93]:
invgamma_ks = []
invgamma_cvm = []

start_time = time.time()
context = get_context("walmart_map.yaml")
for i in range(len(wp_list)):
    # Select data of interest
    wp1 = wp_list[i][0]
    wp2 = wp_list[i][1]
    edge = wp1 + "_" + wp2
    subset = select_data_edge(df, edge)
    #independent variable to plot over
    t_op = subset["operation_time"]

    #Plot against actual data
    length = context.loc[context["edge_id"] == edge]["edge_length"]
    offset= np.min([float(length / (0.55 * 1.1) ), np.min(t_op) - 0.01])
    #offset = 0
    t_obs = t_op - offset

    #prior parameters
    a = 1
    b = 1
    c = 1
    d = 1
    e = 1

    #Bayesian MAP estimate
    tic = time.time()
    alpha_map, beta_map = Inverse_Gamma_bayes3(t_obs, a, b, c, d, e)
    toc = time.time()
    
    #Duration distribution using MAP parameters from Bayesian method
    t_start= 0.01
    t_stop = (np.max(t_op)//5)*5 + 10    #round up to nearest 5 secs
    t_step = 0.01
    t_test = np.arange(t_start,t_stop,t_step)
    p_bayes = Inverse_Gamma(t_test, alpha= alpha_map, beta = beta_map)
    
    #ks test comparison
    ks_bayes = error_ks(t_test+offset,p_bayes, t_op, plot_graph = False)
    if np.isnan(ks_bayes) == False:
        invgamma_ks.append(ks_bayes)
    
    #cvm test comparison
    cvm_bayes = error_cvm(t_test+offset,p_bayes, t_op, plot_graph = False)
    if np.isnan(ks_bayes) == False:
        invgamma_cvm.append(cvm_bayes)
    
    print(i, ks_bayes, cvm_bayes, toc-tic, "secs")

stop_time = time.time()

print("KS:", np.mean(invgamma_ks),"CVM:", np.mean(invgamma_cvm))
print("Total time:", stop_time - start_time)
    

0 0.1903925974589527 0.008851320092814351 0.9332249164581299 secs
1 0.15935165969747206 0.006396334655088125 0.42586278915405273 secs
2 0.11443813110420428 0.02680234896076952 1.5472123622894287 secs
3 0.12501360312964227 0.00143861956280769 1.2382664680480957 secs
4 0.16214657028319235 0.005078765311760171 0.8757915496826172 secs
5 0.19499129014121458 0.007592105274235748 0.39354801177978516 secs
6 0.1075654825877283 0.01614317947781895 13.374800682067871 secs
7 0.15031223267484545 0.007821236756927663 0.7462937831878662 secs
8 0.1323260161127141 0.006295722543572345 0.5565543174743652 secs
9 0.08605952814380136 0.0061145875049947625 6.617982625961304 secs
KS: 0.1422597111333767 CVM: 0.009253422014078933
Total time: 27.490989446640015


In [94]:
ind = 8
gof["model"][ind] = "exact invgamma offset"
gof["KS"][ind] = np.mean(invgamma_ks)
gof["CVM"][ind] = np.mean(invgamma_cvm)
gof["time (s)"][ind] = stop_time - start_time
gof.head(10)

,model,KS,CVM,time (s)
0,lognorm,0.324026,0.138919,20.8508
1,lognorm offset,0.042664,0.0620705,52.5546
2,invgauss,0.723636,2.42913,12.611
3,invgauss offset,0.614694,0.273621,12.6268
4,approx invgamma,0.197484,0.089863,0.171541
5,approx invgamma offset,0.122744,0.0113857,0.905008
6,approx invgamma cutoff,0.0576373,0.113271,0.163584
7,exact invgamma,0.616321,1.36786,26.2634
8,exact invgamma offset,0.14226,0.00925342,27.491
9,NaN,NaN,NaN,NaN


## 5c) Exact Invgamma - cutoff

In [95]:
invgamma_ks = []
invgamma_cvm = []

start_time = time.time()
for i in range(len(wp_list)):
    # Select data of interest
    wp1 = wp_list[i][0]
    wp2 = wp_list[i][1]
    edge = wp1 + "_" + wp2
    subset = select_data_edge(df, edge)
    #independent variable to plot over
    t_op = subset["operation_time"]

    #Plot against actual data
    #offset = np.min(t_op) - 0.001
    offset = 0
    t_obs = t_op - offset

    #prior parameters
    a = 1
    b = 1
    c = 1
    d = 1
    e = 1

    #Bayesian MAP estimate
    tic = time.time()
    alpha_map, beta_map = Inverse_Gamma_bayes3(t_obs, a, b, c, d, e)
    toc = time.time()
    
    #Duration distribution using MAP parameters from Bayesian method
    t_start= 0.01
    t_stop = (np.max(t_op)//5)*5 + 10    #round up to nearest 5 secs
    t_step = 0.01
    t_test = np.arange(t_start,t_stop,t_step)
    p_bayes = Inverse_Gamma(t_test, alpha= alpha_map, beta = beta_map)
    
    p_bayes[t_test<np.min(t_op)] = 0
    #normalise
    spacing = t_test[1]-t_test[0]
    norm_const = spacing * np.sum(p_bayes)
    p_bayes /= norm_const
    
    #ks test comparison
    ks_bayes = error_ks(t_test+offset,p_bayes, t_op, plot_graph = False)
    if np.isnan(ks_bayes) == False:
        invgamma_ks.append(ks_bayes)
    
    #cvm test comparison
    cvm_bayes = error_cvm(t_test+offset,p_bayes, t_op, plot_graph = False)
    if np.isnan(ks_bayes) == False:
        invgamma_cvm.append(cvm_bayes)
    
    print(i, ks_bayes, cvm_bayes, toc-tic, "secs")

stop_time = time.time()

print("KS:", np.mean(invgamma_ks),"CVM:", np.mean(invgamma_cvm))
print("Total time:", stop_time - start_time)
    

0 0.0 1.569358781760596 1.0467736721038818 secs
1 0.0 1.6473123414963509 0.43909692764282227 secs
2 0.0 2.2358695116250606 1.5350604057312012 secs
3 0.0 1.581896926592399 1.2821834087371826 secs
4 0.0 1.7020608629259015 0.8956010341644287 secs
5 0.0 1.8371212378381203 0.37599897384643555 secs
6 0.048929491768678446 0.4035322299314196 12.26877236366272 secs
7 0.004454711716528607 1.1921432830127272 0.7038538455963135 secs
8 0.0 1.8823540778633423 0.5637741088867188 secs
9 0.02918964598389603 0.558553251650713 6.621873140335083 secs
KS: 0.008257384946910308 CVM: 1.461020250469663
Total time: 25.79282283782959


In [64]:
ind = 9
gof["model"][ind] = "exact invgamma cutoff"
gof["KS"][ind] = np.mean(invgamma_ks)
gof["CVM"][ind] = np.mean(invgamma_cvm)
gof["time (s)"][ind] = stop_time - start_time
gof.head(10)

,model,KS,CVM,time (s)
0,lognorm,0.187188,0.0819397,139.681
1,lognorm offset,0.0837362,0.0220138,522.128
2,invgauss,0.32508,0.235623,73.0167
3,invgauss offset,0.137153,0.0355159,55.0786
4,approx invgamma,0.140192,0.0673382,0.566297
5,approx invgamma offset,0.477864,0.287487,0.150368
6,approx invgamma cutoff,0.0626242,0.0959327,0.544655
7,exact invgamma,0.279556,0.173366,149.316
8,exact invgamma offset,0.439169,0.270414,75.3198
9,exact invgamma cutoff,0.0728468,0.254556,149.55


# 6) Save Dataframe


In [65]:
#Save to excel
gof.to_excel("GOF_10edges.xlsx")
print("done")

done
